In [1]:
%reset #clear the ipython workspace variables

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Don't know how to reset  #clear, please run `%reset?` for details
Don't know how to reset  the, please run `%reset?` for details
Don't know how to reset  ipython, please run `%reset?` for details
Don't know how to reset  workspace, please run `%reset?` for details
Don't know how to reset  variables, please run `%reset?` for details


In [2]:
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import sys
from pandas import DataFrame
import math
import gc
from contextlib import redirect_stdout
import os

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


In [3]:
#data is on google drive so need to load
from google.colab import drive
drive.mount('/content/drive')
#this line needs to be changed depending on file structure of google drive
os.chdir('/content/drive/MyDrive/WVU_GradStudents_ShareFolder/Jacob_Research/New LSTM Crack')

Mounted at /content/drive


In [4]:
#File_name=['1600N_HT_573K.csv', '1700N_HT_573K.csv']
File_name = glob.glob('./Test/HeatTreated/1600N_HT_573K.csv')
print(File_name)
df = pd.concat([pd.read_csv(f) for f in File_name])
print(df)

['./Test/HeatTreated/1600N_HT_573K.csv']
        # Temp   Stress Intensity
0        573.0       8.569023e+09
1        573.0       5.699948e+09
2        573.0       7.606639e+09
3        573.0       6.484521e+09
4        573.0       5.479219e+09
...        ...                ...
267126   573.0       9.774030e+12
267127   573.0       1.003115e+13
267128   573.0       9.812983e+12
267129   573.0       9.769964e+12
267130   573.0       9.728237e+12

[267131 rows x 2 columns]


In [6]:
crack_series = df.iloc[:,1].to_numpy().reshape((len(df),1))
force_series = df.iloc[:,2].to_numpy().reshape((len(df),1))
temp_series = df.iloc[:,3].to_numpy().reshape((len(df),1))

#let's plot the data to see what is going on
fig, (ax1,ax2,ax3) = plt.subplots(3,1,sharex=True)
ax1.plot(crack_series[:,], '-', label='Time') # 
ax2.plot(force_series[:,], '-', label='Force')
ax3.plot(temp_series[:,], '-', label='Temperature')
plt.show()

IndexError: ignored

In [ ]:
#Calculate the stress intensity
#the geometry of the specimen had to be updated on 5/19/21
stress_intensity = []
specimen_width = 34.63e-3 #meters #40.8 mm new width 34.63mm
thickness = 0.4572e-3 #meters thickness of sample 0.4572mm
a0 = 10e-3 #initial crack length is 10mm
crossSectArea = (specimen_width-a0)*thickness #2.6388 meters 

#looks like you are using the polynomial method derived from FEA to determine stress intensity
for i in range(len(crack_series)):
  term1 = (force_series[i] / crossSectArea) * sqrt(3.14 * crack_series[i])
  term2 = 8.574
  term3 = 10.365 * (crack_series[i] / specimen_width)
  term4 = 5.499 * ((crack_series[i] / specimen_width)**2)
  intensity = term1*(term2 - term3 + term4)
  stress_intensity.append(intensity)
  if i % 100000 == 0:
    print(i)

plt.plot(stress_intensity)

In [ ]:
#Note had to remove 1600N_25 because stress intensity was a huge outlier
stress_intensity = np.asarray(stress_intensity).reshape(len(df),1)
series= np.hstack((crack_series, force_series, temp_series))

In [ ]:
print(series.shape)

In [ ]:
#series = np.delete(series, 0, axis = 1)

In [ ]:
  np.savetxt('./Predict/HeatTreated/../../{}'.format(File_name[0]), series, delimiter=",", header='Temp, Stress Intensity')